In [13]:
# Import libraries

import numpy as np
import pandas as pd

import sqlite3

import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

import joblib
from pathlib import Path

### Load the dataset

In [3]:
# Load data from an SQL view
conn = sqlite3.connect("../data/telco_churn.db")
df = pd.read_sql_query("SELECT * FROM vw_churn_training_dataset", conn)
conn.close()

In [5]:
print(df.shape)


(7043, 25)


In [6]:
print(df.head(3))

  customer_id  gender  senior_citizen partner dependents        country  \
0  3668-QPYBK    Male               0      No         No  United States   
1  9237-HQITU  Female               0      No        Yes  United States   
2  9305-CDSKC  Female               0      No        Yes  United States   

        state   contract_type paperless_billing    payment_method  ...  \
0  California  Month-to-month               Yes      Mailed check  ...   
1  California  Month-to-month               Yes  Electronic check  ...   
2  California  Month-to-month               Yes  Electronic check  ...   

  device_protection tech_support streaming_tv streaming_movies tenure_months  \
0                No           No           No               No             2   
1                No           No           No               No             2   
2               Yes           No          Yes              Yes             8   

  monthly_charges total_charges    cltv churn_target  snapshot_date  
0          

### Define target and feature columns

We will NOT use:

- `customer_id` (identifier)
- `snapshot_date` (constant)
- `churn_target` as feature (it’s the target)

In [7]:
TARGET = "churn_target"
DROP_COLS = ["customer_id", "snapshot_date"]

X = df.drop(columns=DROP_COLS + [TARGET])
y = df[TARGET].astype(int)

In [8]:
print(X.shape)

(7043, 22)


In [9]:
print(y.value_counts())

churn_target
0    5174
1    1869
Name: count, dtype: int64


In [10]:
# Quick missing-value check
nulls = X.isna().sum().sort_values(ascending=False)
print(nulls[nulls > 0].head(20))

total_charges    11
dtype: int64


### Feature Engineering

In [11]:
# Create a copy
X_fe = X.copy()

In [14]:
# safe avg monthly spend
X_fe["avg_monthly_spend"] = np.where(
    X_fe["tenure_months"] > 0,
    X_fe["total_charges"] / X_fe["tenure_months"],
    0.0
)

In [15]:
# tenure band
X_fe["tenure_band"] = pd.cut(
    X_fe["tenure_months"],
    bins=[-1, 12, 24, 48, 1200],
    labels=["0-12", "13-24", "25-48", "49+"]
).astype(str)

In [16]:
# addon count (Yes/No columns)
addon_cols = [
    "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies"
]
X_fe["addon_count"] = sum((X_fe[c] == "Yes").astype(int) for c in addon_cols)

In [17]:
X_fe[["tenure_months","total_charges","avg_monthly_spend","tenure_band","addon_count"]].head()


,tenure_months,total_charges,avg_monthly_spend,tenure_band,addon_count
0,2,108.15,54.075000,0-12,2
1,2,151.65,75.825000,0-12,0
2,8,820.50,102.562500,0-12,3
3,28,3046.05,108.787500,25-48,4
4,49,5036.30,102.781633,49+,4


### Build the Sklearn preprocessing pipeline

In [23]:
# Split the data in train and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [24]:
# Custom feature engineering transformer
class TelecomFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.addon_cols = [
            "online_security", "online_backup", "device_protection",
            "tech_support", "streaming_tv", "streaming_movies"
        ]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # ensure numeric
        X["tenure_months"] = pd.to_numeric(X["tenure_months"], errors="coerce")
        X["total_charges"] = pd.to_numeric(X["total_charges"], errors="coerce")
        X["monthly_charges"] = pd.to_numeric(X["monthly_charges"], errors="coerce")
        X["cltv"] = pd.to_numeric(X["cltv"], errors="coerce")

        # engineered features
        X["avg_monthly_spend"] = np.where(
            X["tenure_months"].fillna(0) > 0,
            X["total_charges"] / X["tenure_months"].replace({0: np.nan}),
            0.0
        )
        X["avg_monthly_spend"] = X["avg_monthly_spend"].replace([np.inf, -np.inf], np.nan)

        X["tenure_band"] = pd.cut(
            X["tenure_months"].fillna(0),
            bins=[-1, 12, 24, 48, 1200],
            labels=["0-12", "13-24", "25-48", "49+"]
        ).astype(str)

        X["addon_count"] = 0
        for c in self.addon_cols:
            if c in X.columns:
                X["addon_count"] += (X[c] == "Yes").astype(int)

        return X


In [25]:
# Numerical features
numeric_features = [
    "tenure_months", "monthly_charges", "total_charges", "cltv",
    "avg_monthly_spend", "addon_count"
]

# Categorical features
categorical_features = [c for c in X.columns if c not in ["tenure_months","monthly_charges","total_charges","cltv"]]


In [26]:
# Also adding the engineered feature in the categorical features list
categorical_features += ["tenure_band"]

### Pre processing pipeline

In [27]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = Pipeline(steps=[
    ("fe", TelecomFeatureEngineer()),
    ("ct", ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ],
        remainder="drop"
    ))
])

In [28]:
# Fit-transform to confirm it works
X_train_transformed = preprocessor.fit_transform(X_train, y_train)
X_test_transformed = preprocessor.transform(X_test)

In [29]:
print(X_train_transformed.shape)
print(X_test_transformed.shape)

(5634, 54)
(1409, 54)


### Sanity Visuals

In [30]:
tmp = TelecomFeatureEngineer().fit_transform(X.copy())
tmp["churn_target"] = y.values

In [31]:
px.box(tmp, x="churn_target", y="avg_monthly_spend", title="Avg Monthly Spend vs Churn").show()


In [32]:
px.bar(tmp.groupby("tenure_band")["churn_target"].mean().reset_index(),
       x="tenure_band", y="churn_target", title="Churn Rate by Tenure Band").show()

### Save the preprocessor for API use later

In [33]:
Path("../artifacts").mkdir(exist_ok=True)

joblib.dump(preprocessor, "../artifacts/preprocessor.joblib")
print("Saved: artifacts/preprocessor.joblib")

Saved: artifacts/preprocessor.joblib
